# 1. Установка зависимостей и настройка импорта

In [1]:
# Установка зависимостей
!pip install pandas sentence-transformers scikit-learn requests nltk geopy openrouteservice folium osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [3]:
# Импорт необходимых библиотек
import os
import time
import zipfile
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import requests
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import openrouteservice
import folium
import osmnx as ox

# 2. Загрузка и обработка данных из Yandex Geo Reviews Dataset

In [4]:
# Скачиваем данные с Kaggle
# Используем идентификатор набора данных: kyakovlev/yandex-geo-reviews-dataset-2023
!kaggle datasets download -d kyakovlev/yandex-geo-reviews-dataset-2023

# Предположим, что файл называется 'yandex-geo-reviews-dataset-2023.zip'
zip_file = 'yandex-geo-reviews-dataset-2023.zip'

# Распаковываем содержимое zip-файла
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('extracted_data')

# Загружаем CSV-файл в DataFrame
# Замените 'geo-reviews-dataset-2023.csv' на фактическое имя CSV-файла внутри архива, если оно другое
csv_file_path = 'extracted_data/geo-reviews-dataset-2023.csv'
df = pd.read_csv(csv_file_path)

# Вывод списка столбцов для диагностики
print("Колонки в загруженном DataFrame:", df.columns)

Dataset URL: https://www.kaggle.com/datasets/kyakovlev/yandex-geo-reviews-dataset-2023
License(s): other
yandex-geo-reviews-dataset-2023.zip: Skipping, found more recently modified local copy (use --force to force download)
Колонки в загруженном DataFrame: Index(['address', 'name_ru', 'rating', 'rubrics', 'text'], dtype='object')


In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Маппинг категорий
category_mapping = {
    'Музей': 'история',
    'Театр': 'история',
    'Достопримечательность' : 'история',
    'Монастырь' : 'история',
    'Ресторан': 'гастрономия',
    'Кафе': 'гастрономия',
    'Кофейня': 'гастрономия',
    'Развлекательный центр': 'развлечение',
    'Магазин подарков и сувениров':'развлечение',
    'Аквапарк':'развлечение',
    'Клуб досуга':'развлечение'
}

# Применение маппинга категорий
df['category'] = df['rubrics'].map(category_mapping)

# Фильтрация записей, у которых категория не была найдена
df_filtered = df[df['category'].notna()]

# Загрузка необходимых ресурсов NLTK
nltk.download('stopwords')
nltk.download('wordnet')

# Функция предобработки текста
def preprocess_text(text):
    # Приведение текста к нижнему регистру
    text = text.lower()

    # Удаление HTML-тегов, если они есть
    text = re.sub(r'<.*?>', '', text)

    # Удаление ссылок
    text = re.sub(r'http\S+|www\S+', '', text)

    # Удаление цифр и специальных символов
    text = re.sub(r'[^a-zA-Zа-яА-Я\s]', '', text)

    # Токенизация
    words = text.split()

    # Удаление стоп-слов
    stop_words = set(stopwords.words('russian') + stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Лемматизация (приведение слов к начальной форме)
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Объединение обработанных слов обратно в строку
    return ' '.join(words)

# Применение предобработки к каждому отзыву в датасете
df_filtered['text'] = df_filtered['text'].apply(preprocess_text)

# Создание меток релевантности на основе рейтинга
df_filtered['label'] = (df_filtered['rating'] > 4.0).astype(int)

# Объединение описания и категории в единый текст
df_filtered['combined_text'] = df_filtered.apply(lambda row: row['text'] + ' ' + row['category'], axis=1)

# Вывод результатов для проверки
print("Первые несколько строк отфильтрованного DataFrame:")
print(df_filtered.head())

print("\nУникальные категории после фильтрации:")
print(df_filtered['category'].unique())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
<ipython-input-5-12556303aa3a>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['text'] = df_filtered['text'].apply(preprocess_text)
<ipython-input-5-12556303aa3a>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['label'] = (df_filtered['rating'] > 4.0).astype(int)


Первые несколько строк отфильтрованного DataFrame:
                                              address                name_ru  \
6   Воронежская область, Богучарский район, М-4 До...                 У тещи   
13  Краснодарский край, городской округ Сочи, посё...                 Пандок   
14  Краснодарский край, городской округ Сочи, посё...                 Пандок   
22  Ульяновская область, посёлок городского типа Н...                 Ваниль   
42  Московская область, Одинцовский городской окру...  Шашлычок на Восточном   

    rating   rubrics                                               text  \
6      4.0      Кафе  глубинка страны своих проявлениях ассортимент ...   
13     2.0  Ресторан  самый большой плюс это месторасположение набер...   
14     5.0  Ресторан  добрый день сегодня второй посетили дочкой рес...   
22     5.0      Кафе  отличная кухня прекрасный сервис внимательный ...   
42     5.0      Кафе  лучший шашлык звенигородеnприветливый вежливый...   

       category  

<ipython-input-5-12556303aa3a>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['combined_text'] = df_filtered.apply(lambda row: row['text'] + ' ' + row['category'], axis=1)


# 3. Подготовка и настройка данных для нейросети

In [6]:
# Загрузка модели
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

# 4. Обучение модели

In [7]:
# Кодируем вектора
combined_embeddings = model.encode(df_filtered['combined_text'].to_list())

# 5. Определение городов и построение маршрутов 

In [8]:
 # API-ключ OpenRouteService
API_KEY = "5b3ce3597851110001cf62485690c6ae3c7e4f2a9b1e8ca4f5bfabce"

In [9]:
def get_location_with_address(address):
   # Инициализация геокодера
    geolocator = Nominatim(user_agent="geoapi", timeout=10)

    # Получаем координаты точек
    return geolocator.geocode(address)

In [10]:
def find_cities_on_route(start_address, end_address):
    # Получаем координаты точек
    start_location = get_location_with_address(start_address)
    end_location = get_location_with_address(end_address)

    if not start_location or not end_location:
        raise ValueError("Не удалось получить координаты для одного из адресов.")

    print(f"Стартовые координаты: {start_location.latitude}, {start_location.longitude}")
    print(f"Конечные координаты: {end_location.latitude}, {end_location.longitude}")

    # Создание клиента ORS
    client = openrouteservice.Client(key=API_KEY)

    # Построение маршрута (автомобильный)
    route = client.directions(
        coordinates=[(start_location.longitude, start_location.latitude),
                     (end_location.longitude, end_location.latitude)],
        profile="driving-car",
        format="geojson"
    )

    # Извлечение координат маршрута
    route_coords = [(point[1], point[0]) for point in route["features"][0]["geometry"]["coordinates"]]

    # Инициализация геокодера
    geolocator = Nominatim(user_agent="geoapi", timeout=10)

    # Список для хранения городов на маршруте
    cities_on_route = set()

    # Кэш для координат городов
    city_cache = {}

    # Проход по координатам маршрута, берём каждую 30-ю точку
    for i in range(0, len(route_coords), 30):
        coord = route_coords[i]
        try:
            location = geolocator.reverse(coord, exactly_one=True)
            if location:
                address = location.raw['address']
                city = address.get('city') or address.get('town') or address.get('village')
                if city and city not in cities_on_route:
                    # Проверка расстояния от точки маршрута до центра города
                    if city not in city_cache:
                        city_location = geolocator.geocode(city)
                        if city_location:
                            city_cache[city] = (city_location.latitude, city_location.longitude)
                    if city in city_cache:
                        city_coords = city_cache[city]
                        distance = geodesic(coord, city_coords).km
                        if distance <= 30:  # Радиус поиска
                            cities_on_route.add((city, city_coords))
        except Exception as e:
            print(f"Ошибка геокодирования: {e}")
        time.sleep(0.1)  # Уменьшите задержку между запросами

    # Возвращаем список городов, а также стартовые и конечные координаты
    return list(cities_on_route), (start_location.latitude, start_location.longitude), (end_location.latitude, end_location.longitude)

In [11]:
def create_route_map(start_location, end_location, cities_on_route, start_address, end_address):
    # Создание клиента ORS
    client = openrouteservice.Client(key=API_KEY)

    # Подготовка списка координат для маршрута
    waypoints = [(start_location[1], start_location[0])]  # Переворачиваем координаты для ORS
    waypoints.extend([(city[1][1], city[1][0]) for city in cities_on_route])  # Добавляем города
    waypoints.append((end_location[1], end_location[0]))  # Конечная точка

    # Построение маршрута (автомобильный)
    route = client.directions(
        coordinates=waypoints,
        profile="driving-car",
        format="geojson"
    )

    # Извлечение координат маршрута
    route_coords = [(point[1], point[0]) for point in route["features"][0]["geometry"]["coordinates"]]

    # Создание карты с центром в стартовой точке
    m = folium.Map(location=start_location, zoom_start=6)

    # Добавление маркеров
    folium.Marker(start_location,
        popup=f"<b>Старт:</b> {start_address}",
        icon=folium.Icon(color="green", icon="play")).add_to(m)

    for city_name, city_coords in cities_on_route:
        folium.Marker(city_coords,
            popup=f"<b>Город:</b> {city_name}",
            icon=folium.Icon(color="blue", icon="info-sign")).add_to(m)

    folium.Marker(end_location,
        popup=f"<b>Финиш:</b> {end_address}",
        icon=folium.Icon(color="red", icon="stop")).add_to(m)

    # Добавление маршрута на карту
    folium.PolyLine(route_coords, color="blue", weight=5, opacity=0.7, tooltip="Маршрут").add_to(m)

    # Сохранение карты в файл
    m.save("route_map.html")

    return m

# 6. Генерация рекомендаций на основе модели

In [12]:
def get_top_places(query_embedding, cities_on_route, n_per_city=10):
    if len(query_embedding.shape) == 1:
        query_embedding = query_embedding.reshape(1, -1)

    similarity_scores = cosine_similarity(query_embedding, combined_embeddings).flatten()
    top_indices = similarity_scores.argsort()[::-1]  # Отсортировать по убыванию
    top_places = df_filtered.iloc[top_indices]

    # Отладочный вывод количества всех топ мест
    print(f"Отладка: количество всех топ мест отобранных по косинусному сходству: {len(top_places)}\n")

    recommendations_by_city = {}
    for city_info in cities_on_route:
        city_name = city_info[0].strip().lower()

        # Отладочный вывод
        print(f"Ищем места для города: {city_name}")

        # Фильтрация по городу
        city_places = top_places[top_places['address'].str.contains(city_name, case=False, na=False)]

        # Отладочный вывод
        if city_places.empty:
            print(f"Нет совпадений для города: {city_name}")
        else:
            print(f"Найдено {len(city_places)} совпадений для города: {city_name}")

        if not city_places.empty:
            # Отбор топ мест
            recommendations_by_city[city_name] = city_places.head(n_per_city)[['name_ru', 'address', 'rating', 'category']]

    return recommendations_by_city

In [16]:
def display_top_places(recommendations_by_city):
    print("🏆 Топовые места для вашего запроса: 🏆\n")

    for city, top_places in recommendations_by_city.items():
        print(f"🌆 Город: {city.title()}\n")

        for index, place in top_places.iterrows():
            name = place['name_ru']
            rating = place['rating']
            category = place['category']
            address = place['address']

            print(f"📍 Место: {name}")
            print(f"⭐ Рейтинг: {rating}")
            print(f"🔖 Категория: {category}")
            print(f"📫 Адрес: {address}\n")
            print("-" * 40)

        print("\n" + "=" * 50 + "\n")

# 7. Предоставление тестовых данных и вывод результатов

In [ ]:
# Ввод адресов для построения маршрута
start_address = "Москва, Россия"
end_address = "Казань, Россия"

# Поиск городов по маршруту
cities, start_coords, end_coords = find_cities_on_route(start_address, end_address)

# Смотрим результат выполнения поиска по городам
print(cities)

Стартовые координаты: 55.625578, 37.6063916
Конечные координаты: 55.7823547, 49.1242266
[('городской округ Казань', (55.7823547, 49.1242266)), ('Владимир', (56.1288899, 40.4075203)), ('Москва', (55.625578, 37.6063916)), ('Верхний Услон', (55.7685496, 48.9829213)), ('Казань', (55.7823547, 49.1242266))]


In [17]:
# Запрос пользователя
user_query = "Какие музеи посетить и где вкусно поесть?"

# Кодирование пользовательского запроса
user_query_embedding = model.encode(preprocess_text(user_query)).reshape(1, -1)

# Получение топ мест для пользовательского запроса
top_places_user = get_top_places(user_query_embedding, cities)

# Вывод результатов
print("\nТоп популярных мест для вашего запроса:")
display_top_places(top_places_user)


# Строим и показываем карту
map = create_route_map(start_coords, end_coords, cities, start_address, end_address)
map

Отладка: количество всех топ мест отобранных по косинусному сходству: 41586

Ищем места для города: городской округ казань
Нет совпадений для города: городской округ казань
Ищем места для города: владимир
Найдено 481 совпадений для города: владимир
Ищем места для города: москва
Найдено 7657 совпадений для города: москва
Ищем места для города: верхний услон
Найдено 1 совпадений для города: верхний услон
Ищем места для города: казань
Найдено 1049 совпадений для города: казань

Топ популярных мест для вашего запроса:
🏆 Топовые места для вашего запроса: 🏆

🌆 Город: Владимир

📍 Место: Музей непридуманных историй
⭐ Рейтинг: 5.0
🔖 Категория: история
📫 Адрес: Владимир, улица Ильича, 8

----------------------------------------
📍 Место: Музей ложки
⭐ Рейтинг: 5.0
🔖 Категория: история
📫 Адрес: Владимир, Октябрьская улица, 4

----------------------------------------
📍 Место: Музей ложки
⭐ Рейтинг: 5.0
🔖 Категория: история
📫 Адрес: Владимир, Октябрьская улица, 4

-----------------------------------